In [ ]:
actor_collection.delete_many({})

In [ ]:
'''
여러 page 에서 배우 정보 가져오기 + MongoDB 에 크롤링한 data 저장하기 
'''
# 정규표현식을 사용하지 않고, 문자열 split, indexing, slicing 사용하기

from bs4 import BeautifulSoup
import requests
import pymongo
import re
import pprint
import warnings
warnings.filterwarnings('ignore')

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'  
post_data['period_start'] = '2022-11'  
post_data['gender'] = 'all' 

actor_info_list = []

for idx in range(1, 21):
    post_data['page'] = idx 
    
    response = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(response.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')    

    for idx, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[idx].text.replace(',',''))
        movie_titles = movies[idx].select('li a span')
        movie_title_list = []
        for movie_title in movie_titles:
            # print(movie_title.text)
            movie_title_list.append(movie_title.text)
        # 한 사람의 출연영화 목록    
        # movie_title_list       

        # 한 사람의 상세 정보 : 딕셔너리
        actor_info_dict = {}
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits 
        actor_info_dict['출연영화'] = movie_title_list

        # 상세페이지로 이동해서 정보 가져오기    
        actor_url = 'http://www.cine21.com' + actor.select_one('a').attrs['href']    
        actor_response = requests.get(actor_url)
        actor_soup = BeautifulSoup(actor_response.content, 'html.parser')
        default_info = actor_soup.select_one('ul.default_info')
        actor_detail = default_info.select('li')
        for actor_item in actor_detail:  
            actor_item = str(actor_item)
            elem = actor_item.split('<')
            actor_item_key = elem[2][elem[2].find('>')+1:]
            actor_item_value = elem[3][elem[3].find('>')+1:]
            #  한 사람의 상세정보를 딕셔너리에 추가하기
            actor_info_dict[actor_item_key] = actor_item_value     
       
        # 한 사람의 상세정보를 리스트에 추가하기
        #  ㄴ 반복적으로 실행하면 여러 사람들의 상세정보가 리스트에 추가됨
        actor_info_list.append(actor_info_dict)

actor_collection.insert_many(actor_info_list)        

In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re
import pprint
import warnings
warnings.filterwarnings('ignore')

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

In [ ]:
actor_collection.find_one({})

In [ ]:
# limit(3) : 3 개의 document 만 조회하기
actor_documents = actor_collection.find({}).limit(3)

for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
mongodb 에서 key(field) 이름 변경하기  <-- $rename
  ㄴ update_one() / update_many() 함수 실행
'''

# 다른 이름  --> 다른이름
actor_collection.update_many({}, {'$rename' : {'다른 이름':'다른이름'}})

In [ ]:
# limit(3) : 3 개의 document 만 조회하기
actor_documents = actor_collection.find({}).limit(3)

for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# collection 객체와 연결된 변수 이름 변경하기
actor = actor_collection

In [ ]:
'''
조회하면서 정렬하기 - find() / sort(정렬기준 key)
'''
actor_documents = actor.find({}).sort('생년월일').limit(20)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
조회하면서 정렬하기 - find() / sort(정렬기준 key) 
  ㄴ 내림차순 : pymongo.DESCENDING 속성을 지정함
'''
actor_documents = actor.find({}).sort('생년월일', pymongo.DESCENDING).limit(20)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
특정 key(field) 가 있는지 조회하기 - find() / $exists
'''
# 특기 정보가 있는 배우들만 조회하기
actor_documents = actor.find({'특기': {'$exists' : True}})
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
특기 정보가 있는 배우들을 흥행지수 순으로 조회하기
'''
actor_documents = \
  actor.find({'특기':{'$exists':True}}).sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
생년월일이 없는 배우들의 이름만 조회하기
'''
actor_documents = \
  actor.find({'생년월일':{'$exists':False}},{'_id':0,'배우이름':1})
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
key(field) 값 범위로 조회하기
'''
# 흥행지수가 10000 이상인 배우들만 조회하기
actor_documents = \
  actor.find({'흥행지수':{'$gte':10000}}).sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# 흥행지수가 2000 미만인 배우들만 조회하기
actor_documents = \
  actor.find({'흥행지수':{'$lt':2000}}).sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# 흥행지수가 10000 이상인 배우들의 이름, 흥행지수, 출연영화만 조회하기
actor_documents = \
  actor.find({'흥행지수':{'$gte':10000}}, {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# 올빼미에 출연한 배우들 중에서
# 흥행지수가 10000 이상인 배우들의 이름, 흥행지수, 출연영화 조회하기
# 흥행지수 내림차순
actor_documents = \
  actor.find({'흥행지수':{'$gte':10000}, '출연영화':'올빼미'}, {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수', pymongo.DESCENDING)

In [ ]:
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# 올빼미에 출연한 배우들 중에서
# 흥행지수가 10000 이상인 배우들의 이름, 흥행지수, 출연영화 조회하기
# 배우이름 오름차순
actor_documents = \
  actor.find({'흥행지수':{'$gte':10000}, '출연영화':'올빼미'}, {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('배우이름')

In [ ]:
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# 올빼미나 영웅에 출연한 배우들의 배우이름, 출연영화, 흥행지수 조회하기
# 배우이름 오름차순
actor_documents = \
  actor.find({'$or':[{'출연영화':'올빼미'},{'출연영화':'영웅'}]}, {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('배우이름')
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
# 흥행지수가 10000 이상이고
# 올빼미나 영웅에 출연한 배우들의 배우이름, 출연영화, 흥행지수 조회하기
# 배우이름 오름차순
actor_documents = \
  actor.find({'흥행지수':{'$gte':10000}, '$or':[{'출연영화':'올빼미'},{'출연영화':'영웅'}]}, {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('배우이름')
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
find() - $nor  
  ㄴ nor <-- not nor
  
{'$nor': [{'흥행지수':{'$lte':1000}}, {'흥행지수':{'$gte':10000}}]}
   └ 두 가지 다 아님   └ 1000 이하 ┘               └ 10000 이상┘
        ㄴ 1000 초과 10000 미만이라는 의미              
                      
'''
actor_documents = \
  actor.find({'$nor': [{'흥행지수':{'$lte':1000}},{'흥행지수':{'$gte':10000}}]},\
             {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
find() - $or

{'$or': [{'흥행지수':{'$lte':1000}},{'흥행지수':{'$gte':10000}}]}
 └ 두 가지 다 해당됨  └ 1000 이하 ┘               └ 10000 이상┘
    ㄴ 1000 이하나 10000 이상이라는 의미   
'''
actor_documents = \
  actor.find({'$or': [{'흥행지수':{'$lte':1000}},{'흥행지수':{'$gte':10000}}]},\
             {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
find() - in, nin
in - 포함됨
nin (not in) - 포함되지 않음
'''

actor_documents = \
  actor.find({'흥행지수':{'$in':[78822, 793, 677, 7968]}},\
             {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수', pymongo.DESCENDING)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
흥행지수가 78822, 793, 677, 7968 가 아닌 
배우들 5 명 조회하기
'''
actor_documents = \
  actor.find({'흥행지수':{'$nin':[78822, 793, 677, 7968]}},\
             {'_id':0,'배우이름':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수', pymongo.DESCENDING).limit(5)
for actor_document in actor_documents:
    pprint.pprint(actor_document)

In [ ]:
'''
흥행지수가 78822, 7968 이 아니고, 10000 이하인 
  ㄴ nor (흥행지수가 78822, 7968 이고, 10000 초과)
  ㄴ {'$nor' : [{'흥행지수':{'$in':[78822,7968]}}, {'흥행지수':{'$gt':10000}}]}
배우들 5 명의 정보를 출력하세요 (_id key 만 제외하고 조회하기)
흥행지수 내림차순
'''
actor_documents = \
  actor.find({'$nor' : [{'흥행지수':{'$in':[78822,7968]}}, {'흥행지수':{'$gt':10000}}]},\
             {'_id':0})\
       .sort('흥행지수', pymongo.DESCENDING).limit(5)
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
흥행지수가 78822, 7968 이 아니고, 10000 이하인 
  ㄴ nor (흥행지수가 78822, 7968 이고, 10000 초과)
  ㄴ {'$nor' : [{'흥행지수':{'$in':[78822,7968]}}, {'흥행지수':{'$gt':10000}}]}
배우들 5 명의 배우이름과 흥행지수를 출력하세요
흥행지수 내림차순
'''
actor_documents = \
  actor.find({'$nor' : [{'흥행지수':{'$in':[78822,7968]}},\
                        {'흥행지수':{'$gt':10000}}]},\
             {'_id':0,'배우이름':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING).limit(5)
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
find() - skip(n) : 검색결과를 n 개 만큼 건너뛰고 조회함 
'''
# 흥행지수가 10000 이상인 배우 10 명 검색하기
# 배우이름하고 흥행지수만 조회
actor_documents = actor.find({'흥행지수':{'$gte':10000}},\
                             {'_id':0,'배우이름':1,'흥행지수':1}).limit(10)
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
.limit(10).skip(2)
.skip(2).limit(10)
  ㄴ 순서는 둘 다 됨
  .skip(2) : 맨 앞 두 명은 제외하고 조회함
             맨 앞 류준열, 유해진이 없음
             김래원 뒤 정경호, 안은진이 추가됨
'''
actor_documents = actor.find({'흥행지수':{'$gte':10000}},\
                             {'_id':0,'배우이름':1,'흥행지수':1}).limit(10).skip(2)
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
find() - list 조회 (출연영화 목록)
'''
# 출연영화 중에 올빼미가 있는 배우 조회하기
# 배우이름, 출연영화, 흥행지수만 조회 - 흥행지수 내림차순
actor_documents = \
  actor.find({'출연영화':'올빼미'}, \
             {'_id':0, '배우이름':1,'출연영화':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING) 
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
출연영화 중에 올빼미나 살인의 추억이 있는 배우 조회하기
  ㄴ  {'$or':[{'출연영화':'올빼미'},{'출연영화':'살인의 추억'}]}
 배우이름, 출연영화, 흥행지수만 조회 - 흥행지수 내림차순
'''
actor_documents = \
  actor.find({'$or':[{'출연영화':'올빼미'},{'출연영화':'살인의 추억'}]}, \
             {'_id':0, '배우이름':1,'출연영화':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING) 
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
$all
출연연화 목록 중에서 첫 번째나 두 번째에
올빼미나 리미트가 있는 배우 조회하기
  ㄴ  {'출연영화':{'$all':['올빼미','리미트']}}
 배우이름, 출연영화, 흥행지수만 조회 - 흥행지수 내림차순
'''
actor_documents = \
  actor.find({'출연영화':{'$all':['리미트', '올빼미']}}, \
             {'_id':0, '배우이름':1,'출연영화':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING) 
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
actor_documents = \
  actor.find({'출연영화':{'$all':['올빼미', '리미트']}}, \
             {'_id':0, '배우이름':1,'출연영화':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING) 
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
- 리스트의 index 번호로 조회하기 -

출연연화 목록 중에서 첫 번째에 올빼미가 있는 배우 조회하기
  ㄴ  {'출연영화.0':'올빼미'}
 배우이름, 출연영화, 흥행지수만 조회 - 흥행지수 내림차순
'''
actor_documents = \
  actor.find({'출연영화.0':'올빼미'}, \
             {'_id':0, '배우이름':1,'출연영화':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING) 
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
리스트 사이즈로 조회하기 - 출연영화 개수
'''
# 출연영화가 4 편인 배우 조회하기
#  ㄴ {'출연영화':{'$size':4}}
# 배우이름, 출연영화, 흥행지수만 조회 - 흥행지수 내림차순
actor_documents = \
  actor.find({'출연영화':{'$size':4}}, \
             {'_id':0, '배우이름':1,'출연영화':1,'흥행지수':1})\
       .sort('흥행지수', pymongo.DESCENDING) 
for actor_document in actor_documents:
    pprint.pprint(actor_document) 

In [ ]:
'''
find() - elemMatch
  ㄴ 적어도 하나 이상의 리스트 요소가 
     여러 조건을 동시에 만족하는 경우
'''
from bs4 import BeautifulSoup
import requests
import pymongo
import re
import pprint
import warnings
warnings.filterwarnings('ignore')

conn = pymongo.MongoClient()
actor_db = conn.cine21
elemmatch_sample = actor_db.sample

In [ ]:
elemmatch_sample.delete_many({})

In [ ]:
elemmatch_sample.insert_many([
    {'results':[82, 85, 88]},  
    {'results':[75, 88, 91]},
    {'results':[75, 88, 89]},
    {'results':[85, 88, 92]}
])

In [ ]:
'''
{'results': {'$gte':90, '$lt':80}}
  ㄴ 90 이상과 80 미만이 동시에 있는 document 조회
  ㄴ 'results': [75, 88, 91]
'''
documents = elemmatch_sample.find({'results': {'$gte':90, '$lt':80}})
for document in documents:
    print(document)

In [ ]:
'''
{'results': {'$gte':90, '$lt':85}}
  ㄴ 90 이상과 85 미만이 동시에 있는 document 조회
  ㄴ 'results': [75, 88, 91]
'''
documents = elemmatch_sample.find({'results': {'$gte':90, '$lt':85}})
for document in documents:
    print(document)

In [ ]:
'''
{'results':{'$elemMatch':{'$gte':85, '$lt':90}}}
  ㄴ 85 이상이거나 90 미만인 data 가 하나라도 있으면 OK
  ㄴ 두 가지 조건을 만족하는 data 가 하나라도 있으면 OK
'''
documents = \
  elemmatch_sample.find({'results':{'$elemMatch':{'$gte':85, '$lt':90}}})
for document in documents:
    print(document)

In [ ]:
'''
{'results':{'$elemMatch':{'$gte':75, '$lt':90}}}
  ㄴ 75 이상이거나 90 미만인 data 가 하나라도 있으면 OK
  ㄴ 두 가지 조건을 만족하는 data 가 하나라도 있으면 OK
'''
documents = \
  elemmatch_sample.find({'results':{'$elemMatch':{'$gte':75, '$lt':90}}})
for document in documents:
    print(document)

In [ ]:
'''
{'results':{'$or':[{'$gte':75}, {'$lt':90}]}}
  ㄴ 이렇게 하면 오류 발생함
documents = \
  elemmatch_sample.find({'results':{'$or':[{'$gte':75}, {'$lt':90}]}})
for document in documents:
    print(document)
'''  
1

In [ ]:
'''
직업이 가수인 배우 중,
흥행지수가 가장 높은 순서로 10 명을 출력하세요
'''
documents =\
  actor.find({'직업':'가수'}).sort('흥행지수',pymongo.DESCENDING).limit(10)
for document in documents:
    pprint.pprint(document)

In [ ]:
documents =\
  actor.find({'직업':'가수'},\
             {'_id':0,'배우이름':1,'직업':1,'흥행지수':1,'출연영화':1})\
       .sort('흥행지수',pymongo.DESCENDING).limit(10)
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
탄생에 출연한 배우들을 
흥행지수가 높은 순으로 10 명 출력하세요
출력 필드 : 배우이름, 출연영화, 흥행지수
'''
documents = actor.find({'출연영화':'탄생'},\
                       {'_id':0,'배우이름':1,'출연영화':1,'흥행지수':1})\
                 .sort('흥행지수', pymongo.DESCENDING).limit(10)
for document in documents:
    pprint.pprint(document)

In [ ]:
'''
MongoDB Index
  ㄴ 검색 성능을 향상시키기 위한 data structure
     index 데이터 구조가 없으면
     collection 에 있는 data 를 하나씩 조회하는 방법으로 검색해야 됨
     
기본 index
  ㄴ 모든 MongoDB 의 collection 은 기본적으로 
     _id Field 를 기반으로 하는 기본 index 가 생성됨
'''

# Single Field Index (단일필드인덱스) 생성하기
# 배우이름 field 기반 단일 index
print(actor.create_index('배우이름'))

# Index 정보 확인
print('Index 정보 :\n',actor.index_information())

'''
key : ('field명', direction)
  ㄴ direction
     ㄴ pymongo.ASCENDING = 1
        pymongo.DESCENDING = -1
        pymongo.TEXT = 'text'  <--  부분문자열검색
                         ㄴ 텍스트 검색을 쉽게 할 수 있도록 함
'''

# Index 삭제하기
# actor collection 에 있는 모든 index 삭제
actor.drop_indexes()

# Index 정보 조회하기
print('Index 정보 :\n',actor.index_information())

# Index 생성하기
actor.create_index('배우이름')

# Index 정보 조회하기
print('Index 정보 :\n',actor.index_information())

# Index 삭제하기
# actor collection 에 있는 특정 index 삭제
actor.drop_index([('배우이름', 1)])

# Index 정보 조회하기
print('Index 정보 :\n',actor.index_information())

# Index 생성하기
actor.create_index('배우이름')
actor.create_index('흥행지수')

# Index 정보 조회하기
print('Index 정보 :\n',actor.index_information())


In [ ]:
actor.create_index('배우이름')
actor.create_index('랭킹')
actor.create_index('흥행지수')

In [ ]:
actor.index_information()

In [ ]:
# direction 도 지정하면서 생성하기 - [] 사용
actor.create_index([('직업', 1)])

In [ ]:
actor.index_information()

In [ ]:
actor.drop_indexes()
actor.index_information()

In [ ]:
actor.create_index([('배우이름',1)])
actor.create_index([('랭킹',pymongo.ASCENDING)])
actor.create_index([('흥행지수',pymongo.DESCENDING)])
actor.create_index([('직업', pymongo.TEXT)])
actor.create_index([('출연영화', -1)])
actor.index_information()

In [ ]:
actor.drop_index([('출연영화', -1)])
actor.index_information()

In [ ]:
# actor.create_index([('출연영화', 'text')])

In [ ]:
actor.index_information()

In [ ]:
for doc in actor.find({},{'_id':0,'배우이름':1,'출연영화':1}).limit(30):
    pprint.pprint(doc)

In [ ]:
documents = actor.find({'배우이름':'유해진'})
for doc in documents:
    pprint.pprint(doc)

In [ ]:
actor.index_information()

In [ ]:
actor.drop_indexes()
actor.index_information()

In [ ]:
# OperationFailure: text index required 
actor.create_index([('직업', 'text')])

In [ ]:
'''
{'$text':{'$search':'가수'}}
  ㄴ 텍스트 기반 검색 : 
     정상적으로 실행하려면 
     direction 이 'text' 인 index 가 생성되어 있어야 함
'''
documents = actor.find({'$text':{'$search':'가수'}})
for doc in documents:
    pprint.pprint(doc)

In [ ]:
documents = actor.find({'직업':'가수'})
for doc in documents:
    pprint.pprint(doc)

In [ ]:
actor.drop_indexes()
actor.index_information()

In [ ]:
actor.create_index([('출연영화', pymongo.TEXT)])

In [ ]:
actor.index_information()

In [ ]:
for document in actor.find({},{'_id':0,'배우이름':1,'출연영화':1}).limit(50):
    pprint.pprint(document)

In [ ]:
for document in actor.find({'$text':{'$search':'수학자'}}):
    pprint.pprint(document)

In [ ]:
for document in actor.find({'$text':{'$search':'용의'}}):
    pprint.pprint(document)

In [ ]:
actor.drop_indexes()
actor.index_information()

In [ ]:
# Compound Field 기반 Index 생성하기 - 복합필드인덱스
# 최대 31 개 Field 로 Index 를 생성할 수 있음
actor.create_index([('출연영화', pymongo.TEXT), ('직업', pymongo.TEXT), ('배우이름', pymongo.ASCENDING),('원어명', pymongo.DESCENDING)])

In [ ]:
actor.index_information()

In [ ]:
for document in actor.find({'$text':{'$search':'용의'}}):
    pprint.pprint(document)

In [ ]:
for document in actor.find({'$text':{'$search':'가수'}}):
    pprint.pprint(document)

In [ ]:
'''
Regular Expression (정규표현식)  
   ㄴ $regex 연산자 사용
   cf) $text + $search 두 개 연산자를 같이 사용함
   
   {'출연영화':'용의'} <-- 이렇게 하려면 영화제목을 정확하게 다 입력해야 함
   {'출연영화':{'$regex':'용의'}}  <-- 이렇게 하면 부분문자열 검색이 가능함
'''
for document in actor.find({'출연영화':{'$regex':'용의'}}):
    pprint.pprint(document) 

In [ ]:
'''
actor_db 에 text_collection 이라는 collection 생성하기
'''

# conn = pymongo.MongoClient()
# actor_db = conn.cine21
text_collection = actor_db.text_collection

In [ ]:
text_collection.insert_many([
    {'name':'Java Language', 'description':'Coffee and cakes', "ranking":1},
    {'name':'Burgar Buns', 'description':'Java Hamburgers', "ranking":2},
    {'name':'Coffee Shop', 'description':'Java coffee', "ranking":3},
    {'name':'Clothes Clothes Clothes', 'description':'Discount clothing', "ranking":4},   
    {'name':'Java Shopping', 'description':'Indonesian goods', "ranking":5}
])

In [ ]:
for document in text_collection.find({},{'_id':0}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({},{'_id':0,'name':1}):
    pprint.pprint(document)

In [ ]:
'''

정규표현식으로 구분문자열을 검색할 때는
index 가 없어도 됨

'Co.+'  :  Co 로 시작하면서 그 뒤에 문자가 하나 이상 있는 문자열
'Co.*'  :  Co 로 시작하면서 그 뒤에 문자가 없거나 하나 이상 있는 문자열

{'name' : {'$regex':'Co.+'}  
   <-- name 이 Co 로 시작하면서 그 뒤에 문자가 하나 이상 있는 문자열
'''
for document in text_collection.find({'name' : {'$regex':'Co.+'}},{'_id':0}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'name' : {'$regex':'Co.*'}},{'_id':0}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'name' : {'$regex':'Sh.+'}},{'_id':0}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'name' : {'$regex':'Sh.*'}},{'_id':0}):
    pprint.pprint(document)

In [ ]:
'''
$text : $search 로 부분문자열 검색할 때는
text index 가 있어야 함

[('name', pymongo.TEXT)]
   ㄴ name field 기반으로 부분문자열 검색이 가능함
'''
text_collection.create_index([('name', pymongo.TEXT)])
text_collection.index_information()

In [ ]:
for document in text_collection.find({'$text':{'$search':'coffee'}}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'$text':{'$search':'java coffee shop'}}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'$text':{'$search':'\"coffee shop\"'}}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'$text':{'$search':'coffee shop'}}):
    pprint.pprint(document)

In [ ]:
'''
대소문자 구분해서 검색하기 : '$caseSensitive'True
'''
for document in text_collection.find({'$text':{'$search':'coffee','$caseSensitive':True}}):
    pprint.pprint(document)

In [ ]:
for document in text_collection.find({'$text':{'$search':'Coffee','$caseSensitive':True}}):
    pprint.pprint(document)

In [ ]:
for document in actor.find({},{'_id':0}).limit(50):
    pprint.pprint(document)    

In [ ]:
'''
중앙대학교 출신 배우들을 
흥행지수가 높은 순으로 20 명 출력하세요
'''
print('중앙대학교 출신 배우 :',\
      len(list(actor.find({'학교':{'$regex':'중앙대학교'}}))),'명')    

for document in actor.find({'학교':{'$regex':'중앙대학교'}})\
                     .sort('흥행지수', pymongo.DESCENDING).limit(20):
    pprint.pprint(document)

In [ ]:
for document in actor.find({'학교':{'$regex':'중앙대학교'}})\
                     .sort('흥행지수', pymongo.DESCENDING).limit(20):
    pprint.pprint(document)

In [ ]:
print('서울예술대학교 출신 배우 :',\
      len(list(actor.find({'학교':{'$regex':'서울예술'}}))),'명')    

for document in actor.find({'학교':{'$regex':'서울예술'}})\
                     .sort('흥행지수', pymongo.DESCENDING).limit(20):
    pprint.pprint(document)

In [ ]:
for document in actor.find({'배우이름':'이정재'}):                     
    pprint.pprint(document)